In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.watchmen import DirectLimitWatchman, PcaLimitWatchman
from utils.watchmen import IsoForestWatchman
from utils.watchmen import LinearPredictWatchman, DeepPredictWatchman

In [4]:
SEED = 585

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid2test_ratio': 0.3,
}
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)

# Preparing watchhouse

In [7]:
watchhouse = {
    0: {  # GhlKasperskyDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
    1: {  # TepHarvardDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
    2: {  # TepKasperskyDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
    3: {  # SwatItrustDataset
        0: DirectLimitWatchman(),
        1: PcaLimitWatchman(),
        2: IsoForestWatchman(),
        3: LinearPredictWatchman(),
        4: DeepPredictWatchman(),
    },
}

In [8]:
for d in datasets:
    for w in watchhouse[d]:
        watchhouse[d][w].load(f'{datasets[d].__class__.__name__}-watchman_{w}')

# Examine

## Throw stones

In [9]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchhouse[d]:
        stones[d][w] = list()

In [10]:
for d in datasets:
    for data, faults, info in tqdm(datasets[d].valid_generator(), desc=f'Detect on validation {d}'):
        for w in watchhouse[d]:
            detect = watchhouse[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on validation 0:   0%|          | 0/14 [00:00<?, ?it/s]

Detect on validation 1:   0%|          | 0/3150 [00:00<?, ?it/s]

Detect on validation 2:   0%|          | 0/146 [00:00<?, ?it/s]

Detect on validation 3:   0%|          | 0/2 [00:00<?, ?it/s]

## Individual results

In [11]:
metrics = ('precision', 'recall', 'f1_score')
results = {str(datasets[d]): pd.DataFrame(columns=metrics) for d in datasets}

In [12]:
for d in datasets:
    for w in tqdm(watchhouse[d], desc=f'Collect stones d{d}'):
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(stones[d][w]):
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        results[str(datasets[d])].loc[str(watchhouse[d][w]), metrics] = exam_paper.mean().values

Collect stones d0:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d1:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d2:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d3:   0%|          | 0/5 [00:00<?, ?it/s]

## Ensembling results

In [13]:
thresholds = (2,)
for threshold in thresholds:
    for d in datasets:
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(tqdm(stones[d][0], desc=f'Collect stones d{d}')):
            opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].sum(axis=1) for w in watchhouse[d]], axis=1)
            detect = (opinions.sum(axis=1) >= threshold).astype('uint8')
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
        results[str(datasets[d])].loc[f'ensemble(threshold={threshold})', metrics] = exam_paper.mean().values

Collect stones d0:   0%|          | 0/14 [00:00<?, ?it/s]

Collect stones d1:   0%|          | 0/3150 [00:00<?, ?it/s]

Collect stones d2:   0%|          | 0/146 [00:00<?, ?it/s]

Collect stones d3:   0%|          | 0/2 [00:00<?, ?it/s]

Collect stones d0:   0%|          | 0/14 [00:00<?, ?it/s]

Collect stones d1:   0%|          | 0/3150 [00:00<?, ?it/s]

Collect stones d2:   0%|          | 0/146 [00:00<?, ?it/s]

Collect stones d3:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
for d in results:
    print(d)
    display(results[d].sort_values(by='f1_score', ascending=False))

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
PcaLimitWatchman(n_features=12),0.410256,0.386905,0.366667
DirectLimitWatchman(n_features=12),0.34305,0.630952,0.366331
ensemble(threshold=2),0.241663,0.613095,0.310873
ensemble(threshold=4),0.222222,0.053571,0.07619
IsoForestWatchman(n_features=12),0.023114,0.827381,0.044873
LinearPredictWatchman(n_features=12),0.008929,0.02381,0.012987
Watchman(n_features=12),NaN,0.0,0.0


TepHarvardDataset(E:\Datasets\TEP\dataverse)


,precision,recall,f1_score
LinearPredictWatchman(n_features=52),0.998884,0.757722,0.792104
ensemble(threshold=2),1.0,0.721551,0.766578
DirectLimitWatchman(n_features=52),1.0,0.651693,0.699804
ensemble(threshold=4),1.0,0.613694,0.668459
PcaLimitWatchman(n_features=52),1.0,0.570655,0.618879
IsoForestWatchman(n_features=52),0.845028,0.493972,0.560941
Watchman(n_features=52),NaN,0.0,0.0


TepKasperskyDataset(E:\Datasets\TEP\kaspersky)


,precision,recall,f1_score
DirectLimitWatchman(n_features=53),0.736203,0.437018,0.413159
ensemble(threshold=4),0.615658,0.336897,0.306021
LinearPredictWatchman(n_features=53),0.341616,0.528525,0.263802
ensemble(threshold=2),0.332358,0.536896,0.260004
PcaLimitWatchman(n_features=53),0.920988,0.221336,0.22772
IsoForestWatchman(n_features=53),0.08879,1.0,0.134426
Watchman(n_features=53),NaN,0.0,0.0


SwatItrustDataset(E:\Datasets\SWaT\datasetA1)


,precision,recall,f1_score
DirectLimitWatchman(n_features=51),0.257732,1.0,0.370309
IsoForestWatchman(n_features=51),0.257732,1.0,0.370309
ensemble(threshold=2),0.257732,1.0,0.370309
ensemble(threshold=4),0.241865,0.806818,0.308252
LinearPredictWatchman(n_features=51),0.75,0.094697,0.147222
PcaLimitWatchman(n_features=51),0.357143,0.056818,0.098039
Watchman(n_features=51),NaN,0.0,0.0
